In [3]:
import cv2
import numpy as np

# Court dimensions in meters
court_width = 10.97
court_length = 23.77

# Load video
video_path = 'video/crop - 1.mov'
cap = cv2.VideoCapture(video_path)


[ WARN:0@9963.854] global /private/var/folders/nz/j6p8yfhx1mv_0grj5xl4650h0000gp/T/abs_d9lyif19nl/croot/opencv-suite_1676472756314/work/modules/videoio/src/cap_gstreamer.cpp (2386) handleMessage OpenCV | GStreamer warning: your GStreamer installation is missing a required plugin
[ WARN:0@9963.854] global /private/var/folders/nz/j6p8yfhx1mv_0grj5xl4650h0000gp/T/abs_d9lyif19nl/croot/opencv-suite_1676472756314/work/modules/videoio/src/cap_gstreamer.cpp (2402) handleMessage OpenCV | GStreamer warning: Embedded video playback halted; module uridecodebin2 reported: Your GStreamer installation is missing a plug-in.
[ WARN:0@9963.854] global /private/var/folders/nz/j6p8yfhx1mv_0grj5xl4650h0000gp/T/abs_d9lyif19nl/croot/opencv-suite_1676472756314/work/modules/videoio/src/cap_gstreamer.cpp (1356) open OpenCV | GStreamer warning: unable to start pipeline
[ WARN:0@9963.854] global /private/var/folders/nz/j6p8yfhx1mv_0grj5xl4650h0000gp/T/abs_d9lyif19nl/croot/opencv-suite_1676472756314/work/modules/v

# V1

In [2]:

# Define color range for line detection (blue tennis court with white lines)
lower_color = np.array([96,106,169], dtype=np.uint8)
upper_color = np.array([171, 152, 152], dtype=np.uint8)

# Function to calculate court coordinates
def calculate_court_coordinates(frame, lines):
    height, width = frame.shape[:2]

    # Transform court dimensions to pixel coordinates
    court_pixel_width = int(width * court_width / court_length)
    court_pixel_height = int(height * court_length / court_width)

    # Warp court corners to get court coordinates
    court_coordinates = np.array([[0, 0], [court_pixel_width, 0],
                                  [court_pixel_width, court_pixel_height],
                                  [0, court_pixel_height]], dtype=np.float32)

    return court_coordinates.squeeze()

# Detect court lines in the video frames
while True:
    # Read frame
    ret, frame = cap.read()
    if not ret:
        break

    # Preprocess the frame
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    mask = cv2.inRange(hsv, lower_color, upper_color)

    # Apply morphological operations to enhance line detection
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)

    # Detect lines using Hough transform
    lines = cv2.HoughLinesP(mask, rho=1, theta=np.pi/180, threshold=50, minLineLength=50, maxLineGap=10)

    if lines is not None:
        # Filter and draw the court lines
        court_lines = []
        for line in lines:
            x1, y1, x2, y2 = line[0]
            if abs(y2 - y1) > abs(x2 - x1):
                court_lines.append([(x1, y1), (x2, y2)])
                cv2.line(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)

        # Calculate court coordinates
        court_coordinates = calculate_court_coordinates(frame, court_lines)

    # Display the frame with detected lines
    cv2.imshow("Video with Court Lines", frame)

    if cv2.waitKey(1) == 27:
        break

# Release video capture
cap.release()
cv2.destroyAllWindows()


# V2

In [4]:
# Define color range for line detection (tennis court blue and line colors)
court_blue_lower = np.array([96, 106, 169], dtype=np.uint8)
court_blue_upper = np.array([116, 126, 189], dtype=np.uint8)
line_color_lower = np.array([151, 132, 132], dtype=np.uint8)
line_color_upper = np.array([191, 172, 172], dtype=np.uint8)

# Function to calculate court coordinates
def calculate_court_coordinates(frame, lines):
    height, width = frame.shape[:2]

    # Transform court dimensions to pixel coordinates
    court_pixel_width = int(width * court_width / court_length)
    court_pixel_height = int(height * court_length / court_width)

    # Warp court corners to get court coordinates
    court_coordinates = np.array([[0, 0], [court_pixel_width, 0],
                                  [court_pixel_width, court_pixel_height],
                                  [0, court_pixel_height]], dtype=np.float32)

    return court_coordinates.squeeze()

# Detect court lines in the video frames
while True:
    # Read frame
    ret, frame = cap.read()
    if not ret:
        break

    # Preprocess the frame
    mask_court = cv2.inRange(frame, court_blue_lower, court_blue_upper)
    mask_lines = cv2.inRange(frame, line_color_lower, line_color_upper)
    mask = cv2.bitwise_or(mask_court, mask_lines)

    # Apply morphological operations to enhance line detection
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)

    # Detect lines using Hough transform
    lines = cv2.HoughLinesP(mask, rho=1, theta=np.pi/180, threshold=50, minLineLength=50, maxLineGap=10)

    if lines is not None:
        # Filter and draw the court lines
        court_lines = []
        for line in lines:
            x1, y1, x2, y2 = line[0]
            if abs(y2 - y1) > abs(x2 - x1):
                court_lines.append([(x1, y1), (x2, y2)])
                cv2.line(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)

        # Calculate court coordinates
        court_coordinates = calculate_court_coordinates(frame, court_lines)

    # Display the frame with detected lines
    cv2.imshow("Video with Court Lines", frame)

    if cv2.waitKey(1) == 27:
        break

# Release video capture
cap.release()
cv2.destroyAllWindows()


: 